In [141]:
import sklearn.datasets
import sklearn.metrics
import pandas as pd
import autosklearn.regression
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import openpyxl

In [151]:
#Import BASEo .csv with pandas
data = pd.read_csv('BASEi.csv', index_col=0)
#Split data into train and test the X is all the columns minus 'Energia GBA' which is the target
X = data.drop(['Energia GBA', 'Clima PBA'], axis=1)
y = data['Energia GBA']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [ ]:
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=1800,
    per_run_time_limit=300,
    tmp_folder='autosklearn_regression_example_tmp' )
#exclude =  {'regressor': ["gradient_boosting"]}
#, include = {'regressor': ["extra_trees", "k_nearest_neighbors"]}, exclude=None
automl.fit(X_train, y_train, dataset_name='Energia')

In [135]:
print(automl.leaderboard())

          rank  ensemble_weight               type      cost   duration
model_id                                                               
355          1             0.10  gradient_boosting  0.048428  27.208896
160          2             0.12  gradient_boosting  0.048625  20.847840
396          3             0.02  gradient_boosting  0.048728  12.183606
303          4             0.08  gradient_boosting  0.049294  38.412038
271          5             0.06  gradient_boosting  0.049311  19.197459
192          6             0.04  gradient_boosting  0.049784  30.982189
286          7             0.02  gradient_boosting  0.049842  14.449529
270          8             0.16  gradient_boosting  0.050113  13.846631
93           9             0.12  gradient_boosting  0.050266  13.965696
332         10             0.04  gradient_boosting  0.050277  24.496376
110         11             0.06  gradient_boosting  0.050329  15.453459
106         12             0.02  gradient_boosting  0.050831   7

In [136]:
#Predict the test set
predictions = automl.predict(X_test)
#Print the mean absolute error
print("Mean absolute error: %.2f" % sklearn.metrics.mean_absolute_error(y_test, predictions))

Mean absolute error: 191.66


In [147]:
#Import xlsx
data = pd.read_excel('sep.xlsx', index_col=0)
#Split data into train and test the X is all the columns minus 'Energia GBA' which is the target
X_sep = data.drop(['Energia GBA', 'Clima PBA'], axis=1)
y_sep = data['Energia GBA']

In [148]:
#Predict the test set
predictions = automl.predict(X_sep)
#Print the mean absolute error
print("Mean absolute error: %.2f" % sklearn.metrics.mean_absolute_error(y_sep, predictions))

predictions = pd.DataFrame(predictions)
#Export predictions
predictions.to_csv('predictions.csv')

Mean absolute error: 141.31


In [149]:

#Import xlsx
data = pd.read_excel('oct.xlsx', index_col=0)
#Split data into train and test the X is all the columns minus 'Energia GBA' which is the target
X_oct = data.drop(['Energia GBA', 'Clima PBA'], axis=1)
X_OCT = X_oct[:24]
y_oct = data['Energia GBA']
y_OCT = y_oct[:24]
X_OCT.tail()

,Hora,Tipo Dia,Energia PBA,Clima GBA,Año,Mes
Fecha,,,,,,
2021-10-01 19:00:00,19,6,1789.6,15.533333,2021,10
2021-10-01 20:00:00,20,6,1979.0,13.966667,2021,10
2021-10-01 21:00:00,21,6,2037.5,12.400000,2021,10
2021-10-01 22:00:00,22,6,2026.0,11.766667,2021,10
2021-10-01 23:00:00,23,6,1936.7,11.133333,2021,10


In [150]:
#Predict the test set
predictions = automl.predict(X_OCT)
#Print the mean absolute error
print("Mean absolute error: %.2f" % sklearn.metrics.mean_absolute_error(y_OCT, predictions))

predictions = pd.DataFrame(predictions)
#Export predictions
predictions.to_csv('predictions.csv')

Mean absolute error: 365.99
